In [1]:
!pip install ultralytics opencv-python-headless


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.2 MB/s eta 0:00:00


In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from google.colab import files
files.upload()


In [ ]:
model = YOLO("yolov8n.pt")
video_path = "road.mp4"
cap = cv2.VideoCapture(video_path)

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

out = cv2.VideoWriter(
    "Traffic_monitoring.mp4",
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps,
    (width, height)
)


In [ ]:
frame_memory = defaultdict(int)
CONFIRM_FRAMES = 3
risk_zone = (
    int(width * 0.35),
    int(height * 0.6),
    int(width * 0.65),
    height
)


In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break


    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    brightness = np.mean(gray)
    scene = "Night" if brightness < 80 else "Day"


    results = model(frame, verbose=False)[0]

    vehicle_count = 0
    person_in_risk = False

    cv2.rectangle(
        frame,
        (risk_zone[0], risk_zone[1]),
        (risk_zone[2], risk_zone[3]),
        (0, 0, 255),
        2
    )
    cv2.putText(frame, "CRITICAL ZONE",
                (risk_zone[0], risk_zone[1] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                (0, 0, 255), 2)

    for box, cls, conf in zip(
        results.boxes.xyxy,
        results.boxes.cls,
        results.boxes.conf
    ):
        label = results.names[int(cls)]
        confidence = float(conf)

        if confidence < 0.4:
            continue

        x1, y1, x2, y2 = map(int, box)
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2


        key = (label, x1 // 50, y1 // 50)
        frame_memory[key] += 1
        if frame_memory[key] < CONFIRM_FRAMES:
            continue


        color = (0, 255, 0)

        if label == "person":
            color = (0, 0, 255)
            if (risk_zone[0] < center_x < risk_zone[2] and
                risk_zone[1] < center_y < risk_zone[3]):
                person_in_risk = True

        elif label in ["car", "bus", "truck", "motorcycle"]:
            vehicle_count += 1

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, label,
                    (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    color, 2)

    if vehicle_count <= 5:
        traffic_state = "LOW"
    elif vehicle_count <= 12:
        traffic_state = "MEDIUM"
    else:
        traffic_state = "HIGH"

    if person_in_risk:
        cv2.putText(frame, "PEDESTRIAN RISK ALERT",
                    (30, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (0, 0, 255), 3)

    cv2.putText(frame, f"Traffic: {traffic_state}",
                (30, height - 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                (255, 255, 0), 2)

    cv2.putText(frame, f"Scene: {scene}",
                (30, height - 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                (255, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()

print("Output saved as Traffic_monitoring.mp4")


In [ ]:
%%writefile README.md
# Traffic Scene Awareness using Computer Vision

This project implements a computer vision pipeline to analyze traffic videos using OpenCV and a pre-trained YOLOv8 model.

## Features
- Traffic object detection (vehicles & pedestrians)
- Pedestrian risk zone awareness
- Temporal filtering to reduce false positives
- Traffic density estimation
- Day/Night scene awareness

## Tech Stack
- Python
- OpenCV
- YOLOv8 (Ultralytics)
- Google Colab

## Note
This project focuses on perception and situational awareness, not autonomous control.
